In [1]:
import numpy as _np
import matplotlib.pyplot as _plt
import pandas as _pd
import glob as _gl
import pymad8 as _m8
import pybdsim as _bd
import beamTracking
import beamWire
from scipy.optimize import curve_fit

Welcome to JupyROOT 6.29/01


In [2]:
date = "2023_05_15"
save_name_prefix = "plots/BeamWire/" + date + "_T20_plot"

In [3]:
#beamWire.analyticConvolution()

In [4]:
paramdict = {'x' : {'mean': 0, 'std': 10e-6}, #10e-6
             'xp': {'mean': 0, 'std': 10e-6}, #10e-6
             'y' : {'mean': 0, 'std': 10e-6}, #10e-6
             'yp': {'mean': 0, 'std': 10e-6}, #10e-6
             'z' : {'mean': 0, 'std': 30e-6}, #30e-6
             'DE': {'mean': 0, 'std': 3e-3},  #3e-3
            }

In [5]:
npart = 10000
T_C = _m8.Sim.Track_Collection(14)
T_C.GenerateNtracks(npart, paramdict)
T_C.WriteBdsimTrack('../03_bdsimModel/track_input_bdsim_wire')

In [ ]:
#beamWire.runOneOffset("../03_bdsimModel/T20_for_wire.gmad", npart=npart, diameter=0.5, offsetX=0.1)
beamWire.runScanOffset('../03_bdsimModel/T20_for_wire.gmad', npart=npart, diameter=0.5, offsetXmin=-0.5, offsetXmax=0.5, nbpts=21)

In [ ]:
OFFSETS, NPHOTONS = beamWire.countPhotonsAllFiles('T20_for_wire')

In [ ]:
beamWire.PlotConvolutionExample()
_plt.savefig("{}_convolution_example".format(save_name_prefix), dpi="figure", bbox_inches='tight', facecolor="#FF000000")

In [ ]:
beamWire.PlotConvolution(OFFSETS, NPHOTONS)
_plt.savefig("{}_photon_beam_scan_and_convolution_fit".format(save_name_prefix), dpi="figure", bbox_inches='tight', facecolor="#FF000000")

In [ ]:
_plt.rcParams['font.size'] = 17
fig, ax = _plt.subplots(1,1,figsize=(9,6))
fig.tight_layout()
_plt.plot(OFFSETS, _np.gradient(NPHOTONS, OFFSETS), label='gradient from array')
popt, pcov = curve_fit(beamWire.func_conv, OFFSETS, NPHOTONS, p0=[2e3, 50e-3, 0, 250e-3])
X = _np.linspace(-0.5, 0.5, 500)
_plt.plot(X, _np.gradient(beamWire.func_conv(X, A=popt[0], sigma=popt[1], mu=popt[2], R=popt[3]), X), label='gradient from function')

_plt.xlabel('Offset from center of pipe [mm]')
_plt.ylabel('$\frac{dN}{dx}$')

_plt.savefig("{}_derivative_of_photon_beam_profile".format(save_name_prefix), dpi="figure", bbox_inches='tight', facecolor="#FF000000")

In [ ]:
beamWire.analysis("../04_dataLocal/T20_for_wire_0.root")

In [ ]:
beamWire.plot_hist("../06_analysis/T20_for_wire_0_hist.root", "PHOTONS_E", steps=False, errorbars=True)
_plt.ticklabel_format(axis='both', style='sci', scilimits=(1,8))
_plt.savefig("{}_photon_beam_X_profile".format(save_name_prefix), dpi="figure", bbox_inches='tight', facecolor="#FF000000")